# README
- This notebook includes code to run searches against xml documents 
- Primarily tested for results from arxiv.org and Google Alerts RSS 

TO DO:
- create functions similar to github/twitter to return usable DFs 

XML files (e.g. Arxiv)

In [ ]:
#imports 
import urllib, urllib.request
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
#open url and return it as a local xml file 
url = 'https://export.arxiv.org/api/query?search_query=%22music+collection%22&searchtype=abstract&abstracts=show&order=-announced_date_firs'
data = urllib.request.urlopen(url)
print(data.read().decode('utf-8'))
file = urllib.request.urlretrieve(url, 'arxiv.xml')

In [ ]:
#parse xml file and extract needed info into df 
with open('arxiv.xml', 'r') as f:
	file = f.read() 
 
soup = BeautifulSoup(file, 'lxml')
entries = [entry for entry in soup.find_all('entry')]
titles = [title.text.strip() for title in soup.find_all('title')]
summaries = [summary.text.strip() for summary in soup.find_all('summary')]
links = [link.get('href') for link in soup.find_all('link', href=True, type="text/html")]
all_authors = []
for entry in entries:
	authors = [author.text.strip() for author in entry.find_all('name')]
	all_authors.append(authors)

arxiv_df = pd.DataFrame(columns=['Title', 'Summary', 'Link', 'Authors'])
arxiv_df['Title'] = pd.Series(titles[1:])
arxiv_df['Summary'] = pd.Series(summaries)
arxiv_df['Link'] = pd.Series(links)
arxiv_df['Authors'] = pd.Series(all_authors)
arxiv_df

Google Search RSS

In [ ]:
#open url and return local xml file 
url = 'https://www.google.com/alerts/feeds/11458941541514212856/10321436774246146597'
data = urllib.request.urlopen(url)
print(data.read().decode('utf-8'))
file = urllib.request.urlretrieve(url, 'google_news.xml')

In [ ]:
#parse xml file and extract info into df 

with open('google_news.xml', 'r') as f:
	file = f.read() 
 
soup = BeautifulSoup(file, 'lxml')
date = [date for date in soup.find_all('published')]
titles = [title.text.strip() for title in soup.find_all('title')]
content = [summary.text.strip() for summary in soup.find_all('content')]
links = [link.get('href') for link in soup.find_all('link', href=True)]

google_df = pd.DataFrame(columns=['Date', 'Title', 'Content', 'Link'])
google_df['Date'] = pd.Series(date).astype('object')
google_df['Title'] = pd.Series(titles).astype('string')
google_df['Content'] = pd.Series(content).astype('string')
google_df['Link'] = pd.Series(links[1:]).astype('string')
google_df